In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm


wanted_exposure = None
initial_investment = 3000
interesting_n_assets = 10


def compute_percentage(value, percentage):
    return value * (1 + (percentage / 100))


def load_dataframe(csv_name):
    df = pd.read_csv(csv_name)
    
    # reasign Tag column related to HODL Fiat Balance
    df.loc[df['HODL Fiat Balance'] < compute_percentage(initial_investment, -15), 'Tag'] = 'bearish'
    df.loc[df['HODL Fiat Balance'] > compute_percentage(initial_investment, 15), 'Tag'] = 'bullish'
    df.loc[(df['HODL Fiat Balance'] >= compute_percentage(initial_investment, -15)) & (df['HODL Fiat Balance'] <= compute_percentage(initial_investment, 15)), 'Tag'] = 'same-price'
    
    # df.loc[df['HODL Fiat Balance'] > 0, 'Tag'] = 'global'

    return df


# Histograms
def plot_profits_histogram(df, exposure, n_assets, tag):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
    fig.set_figwidth(24)

    filtered_df = df[(df["Tag"] == tag) & ((df["Exposure"] == exposure)) & ((df["No. Assets"] == n_assets))]
    hodl_profits = filtered_df['HODL Profit']
    rebalance_profits = filtered_df['Rebalance Profit']
    
    def _plot_axis(ax, period):
        period_df = filtered_df[(filtered_df["Period"] == period)]
        profits = period_df['Rebalance Profit']

        ax.set_xlabel('Profits %')
        ax.hist(profits, bins=20, alpha=0.25, color='green', label='Rebalancing Period ' + period)
        ax.legend(loc='upper right')
        ax.set_title(f'E:{exposure} - Assets:{n_assets} - Tag:{tag} - {period}')

    
    ax1.set_xlabel('Profits %')
    # plt.hist([hodl_profits, rebalance_profits], bins=20, color=['red', 'green'], label=['HODL', 'Rebalancing'])
    ax1.hist(hodl_profits, bins=20, alpha=0.25, color='red', label='HODL')
    ax1.hist(rebalance_profits, bins=20, alpha=0.25, color='green', label='Rebalancing')
    ax1.legend(loc='upper right')
    ax1.set_title(f'E:{exposure} - Assets:{n_assets} - Tag:{tag}')
    
    _plot_axis(ax2, '1h')
    _plot_axis(ax3, '1d')
    _plot_axis(ax4, '1w')
    
    plt.show()


def profits_and_histograms(df):

    best_for = {}

    for tag in sorted(list(df["Tag"].unique())):
        if tag not in best_for:
            best_for[tag] = None

        for exposure in df["Exposure"].unique():
            filtered_df = df[(df["Tag"] == tag) & ((df["Exposure"] == exposure))]
            n_currencies_dfs = {}

            mean_profits = []
            stdev_profits = []

            hodl_profits = []
            stdev_hodl_profits = []

            mean_profits_related_to_hold = []
            stdev_profits_related_to_hold = []

            profits_x = []
            for n in range(12):
                current_df = n_currencies_dfs[n + 1] = filtered_df[filtered_df["No. Assets"] == n + 1]
                if len(current_df) == 0:
                    continue

                mean_profits.append(current_df['Rebalance Profit'].mean())
                stdev_profits.append(current_df['Rebalance Profit'].std())

                hodl_profits.append(current_df['HODL Profit'].mean())
                stdev_hodl_profits.append(current_df['HODL Profit'].std())

                mean_profits_related_to_hold.append(current_df['Profit related to HODL strategy'].mean())
                stdev_profits_related_to_hold.append(current_df['Profit related to HODL strategy'].std())
                profits_x.append(n + 1)


            global_mean_profits = round(sum(mean_profits) / len(mean_profits), 2)

            if best_for[tag] is None or best_for[tag]['global_mean_profits'] < global_mean_profits:
                if wanted_exposure is None or (wanted_exposure is not None and wanted_exposure == exposure):
                    best_for[tag] = {
                        'global_mean_profits': global_mean_profits,
                        'exposure': exposure,
                        'profits_x': profits_x,
                        'mean_profits': mean_profits,
                        'stdev_profits': stdev_profits,
                        'hodl_profits': hodl_profits,
                        'stdev_hodl_profits': stdev_hodl_profits,

                        'mean_profits_related_to_hold': mean_profits_related_to_hold,
                    }


    for tag, data in best_for.items():
        global_mean_profits = data['global_mean_profits']
        exposure = data['exposure']
        profits_x = data['profits_x']
        mean_profits = data['mean_profits']
        stdev_profits = data['stdev_profits']
        hodl_profits = data['hodl_profits']
        stdev_hodl_profits = data['stdev_hodl_profits']
        mean_profits_related_to_hold = data['mean_profits_related_to_hold']
        global_mean_profits = data['global_mean_profits']
        global_mean_profits = data['global_mean_profits']
        global_mean_profits = data['global_mean_profits']
        global_mean_profits = data['global_mean_profits']


        fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
        fig.set_figwidth(24)

        ax1.plot(profits_x, mean_profits, 'g')
        ax1.fill_between(profits_x, np.array(mean_profits) - np.array(stdev_profits), np.array(mean_profits) + np.array(stdev_profits), alpha=0.1)
        ax1.legend(['Rebalance profits'])
        ax1.set_xlabel('No. Assets')
        ax1.set_ylabel('Profits')
        ax1.set_title(f'For {tag} markets, exposure {exposure}, mean {global_mean_profits}%')

        ax2.plot(profits_x, hodl_profits, 'r')
        ax2.fill_between(profits_x, np.array(hodl_profits) - np.array(stdev_hodl_profits), np.array(hodl_profits) + np.array(stdev_hodl_profits), alpha=0.1)
        ax2.legend(['HODL profits'])
        ax2.set_xlabel('No. Assets')
        ax2.set_ylabel('Profits')
        # ax2.set_ylim(min(hodl_profits) - 2, max(hodl_profits) + 2)
        ax2.set_title(f'For {tag} markets, mean {round(sum(hodl_profits) / len(hodl_profits), 2)}%')

        ax3.plot(profits_x, mean_profits_related_to_hold)
        ax3.set_xlabel('No. Assets')
        ax3.set_ylabel('Profits related to HODL')
        ax3.legend(['Profits related to HODL'])
        ax3.set_title(f'For {tag} markets, exposure {exposure}, mean {round(sum(mean_profits_related_to_hold) / len(mean_profits_related_to_hold), 2)}%')

        plt.show()

        plot_profits_histogram(df, exposure, interesting_n_assets, tag)

        for n in range(12):
            # 3.- Tenemos que mostrar un surface graph, con exposure/periods/profits
            # print(f'For {tag} markets {n_currencies}: {profit_related_to_hodl}%')
            current_df = n_currencies_dfs[n + 1]


def text_period_to_number(period):
    if period == '1h':
        return 1
    elif period == '2h':
        return 2
    elif period == '4h':
        return 4
    elif period == '6h':
        return 6
    elif period == '8h':
        return 8
    elif period == '12h':
        return 12
    elif period == '1d':
        return 24
    elif period == '1w':
        return 24 * 7
    elif period == '2w':
        return 24 * 7 * 2
    raise Exception(f'Unkown text period {period}')

    
def number_period_to_text(period):
    text_period = '1h'
    if period == 1:
        text_period = '1h'
    elif period == 2:
        text_period = '2h'
    elif period == 4:
        text_period = '4h'
    elif period == 6:
        text_period = '6h'
    elif period == 8:
        text_period = '8h'
    elif period == 12:
        text_period = '12h'
    elif period == 24:
        text_period = '1d'
    elif period == 24 * 7:
        text_period = '1w'
    elif period == 24 * 7 * 2:
        text_period = '2w'
    else:
        raise Exception(f'Unknown numerical period {period}')
    return text_period


def plot_surfaces(df):
    for n_assets in sorted(list(df["No. Assets"].unique())):
        different_tags = list(df["Tag"].unique())

        fig, axs = plt.subplots(1, len(different_tags), subplot_kw={"projection": "3d"})
        fig.set_figwidth(20)

        for i, tag in enumerate(sorted(different_tags)):
            current_df = df[(df['No. Assets'] == n_assets) & (df['Tag'] == tag)]
            mean_hodl_profit = current_df['HODL Profit'].mean()
            stdev_hodl_profit = current_df['HODL Profit'].std()

            X = []
            Y = []
            Z = []

            best_point = None
            worst_point = None
            
            for exposure in df["Exposure"].unique():
                for period in df["Period"].unique():
                    x = text_period_to_number(period)
                    y = exposure
                    exposure_period_df = current_df[(current_df['Exposure'] == exposure) & (current_df['Period'] == period)]
                    z = exposure_period_df['Rebalance Profit'].mean()
                    z_stdev = exposure_period_df['Rebalance Profit'].std()
                    if np.isnan(z_stdev):
                        z_stdev = 0.0

                    X.append(x)
                    Y.append(y)
                    Z.append(z)
                        
                    if wanted_exposure is not None and wanted_exposure != exposure:
                        continue
                    
                    if best_point is None or best_point[2] < z:
                        best_point = (x, y, z, z_stdev)
                    if worst_point is None or worst_point[2] > z:
                        worst_point = (x, y, z, z_stdev)

            X = np.array(X)
            Y = np.array(Y)
            Z = np.array(Z)
            
            try:
                ax = axs[i]
            except TypeError:
                ax = axs

            ax.view_init(azim=350)
                
            ax.set_title(f'{tag.upper()} MARKET, {n_assets} assets, HODL Profit: {round(mean_hodl_profit, 1)}% ±{round(stdev_hodl_profit, 1)}')
            ax.set_xlabel(f'Period (h)')
            ax.set_ylabel(f'Exposure')
            ax.set_zlabel(f'Profits (%)')
            surf = ax.plot_trisurf(X, Y, Z, cmap=cm.coolwarm, alpha=0.5, linewidth=0, antialiased=True)

            bp = best_point
            best_text_period = number_period_to_text(bp[0])
            ax.plot(*bp[:-1], "ro", markersize=5)
            # add a cross on the floor for better visualization
            ax.plot(bp[0], bp[1], np.min(Z), "rx", alpha=0.3, markersize=5)
            ax.text(0, bp[1], bp[2]*0.9, f'E: {bp[1]}, P: {best_text_period}, {round(bp[2], 1)}% ±{round(bp[3], 1)}', color='red', weight='bold')

            wp = worst_point
            worst_text_period = number_period_to_text(wp[0])
            ax.plot(*wp[:-1], "bo", markersize=5)
            # add a cross on the floor for better visualization
            ax.plot(wp[0], wp[1], np.min(Z), "bx", alpha=0.3, markersize=5)
            ax.text(0, wp[1], wp[2]*0.9, f'E: {wp[1]}, P: {worst_text_period}, {round(wp[2], 1)}% ±{round(wp[3], 1)}', color='blue', weight='bold')

            fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5, location='left')

In [ ]:
profits_and_histograms(load_dataframe('../simulation.csv'))
plot_surfaces(load_dataframe('../simulation.csv'))

profits_and_histograms(load_dataframe('../simulations.csv'))
plot_surfaces(load_dataframe('../simulations.csv'))

# profits_and_histograms(load_dataframe('../simulations.csv'))
# profits_and_histograms(load_dataframe('../standard_simulation_no_sell_if_no_buys.csv'))

# plot_surfaces(load_dataframe('../standard_simulation_no_sell_if_no_buys.csv'))